In [169]:
import copy
# Initialize all the matrices required for the hidden markov models (for testing only)
N = 2
M = 3
T = 4
pi = [0.60, 0.40]
O = [0, 1, 0, 2]
A = [[0.70, 0.30], [0.40, 0.60]]
B = [[0.10, 0.40, 0.50], [0.70, 0.20, 0.10]]

In [170]:
# The Forward algorithm. Alpha-pass
# pi represents probability of initial states
# A is the state transition matrix
# B is the observation probability matrix
# N is the number of distinct states of the markov process
# T is the length of the observation sequence
# M is the number of ovservation symbols
# O is the observation sequence
def alpha_pass(pi, A, B, N, M, T, O):
    # The alpha pass calculates the probability P(O_0, O_1, O_2 ... O_t, X_T = q_i | model)

    # initialize the zeroth alpha-pass
    alpha = []
    for states in range(0, N):
        alpha.append([pi[states] * B[states][O[0]]]);
    
    # starting the DP algorithm here
    for time in range(1, T):

        # for each state
        for states in range(0, N):
            # calculate transition probabilities from each state in time-1 and add it
            prob = 0
            for prev_states in range(0, N):
                prob += (alpha[prev_states][time - 1] * A[prev_states][states]);
            
            # multiply it with probabilities to get the observation symbols
            alpha[states].append(prob * B[states][O[time]])

    return alpha

In [171]:
alpha = alpha_pass(pi, A, B, N, M, T, O)
print(alpha)

[[0.06, 0.06159999999999999, 0.0058, 0.007741999999999998], [0.27999999999999997, 0.0372, 0.02855999999999999, 0.0018875999999999995]]


In [172]:
# The backward algorithm. Beta-pass
# pi represents probability of initial states
# A is the state transition matrix
# B is the observation probability matrix
# N is the number of distinct states of the markov process
# T is the length of the observation sequence
# M is the number of ovservation symbols
# O is the observation sequence
def beta_pass(pi, A, B, N, M, T, O):
    # initialize the beta-pass
    beta = [[1] * T for _ in range(0, N)]
    
    # The algorithm starts here
    for time in range(T - 2, -1, -1):
        
        for states in range(0, N):
            # find the sum of transition probabilities multiplied with the observational probability and the beta-pass of next states
            beta[states][time] = 0
            for next_states in range(0, N):
                beta[states][time] += (A[states][next_states] * B[next_states][O[time + 1]] * beta[next_states][time + 1])
    
    return beta

In [173]:
beta = beta_pass(pi, A, B, N, M, T, O)
print(beta)

[[0.030199999999999998, 0.0812, 0.38, 1], [0.02792, 0.12440000000000001, 0.26, 1]]


In [174]:
# find the most likely state sequence given the observation sequence in the HMM sense
def hmm_sense(pi, A, B, N, M, T, O):
    # calculate alpha and beta pass
    alpha = alpha_pass(pi, A, B, N, M, T, O)
    beta = beta_pass(pi, A, B, N, M, T, O)

    # calculate P(O|model), although we don't need it
    P_O = 0
    for i in range(0, N):
        P_O += alpha[i][T - 1]
    
    # Calculate most likely sequence
    most_likely = ""
    for time in range(0, T):
        mx = 0
        for stat in range(1, N):
            prob1 = (alpha[stat][time] * beta[stat][time])
            prob2 = (alpha[mx][time] * beta[mx][time])
            if prob1 > prob2:
                mx = stat
        most_likely = most_likely + str(mx)

    return most_likely

In [175]:
# find the most likely state sequence given the observation sequence in the Dynamic Programming sense
def dp_sense(pi, A, B, N, M, T, O):
    predecessor_list = [[] for _ in range(0, N)]
    alpha = []
    for states in range(0, N):
        alpha.append([pi[states] * B[states][O[0]]]);
        predecessor_list[states].append(states)
    
    # starting the DP algorithm here
    for time in range(1, T):

        # for each state
        for states in range(0, N):
            # calculate transition probabilities from each state in time-1 and add it
            prob_mx = 0
            target = 0
            for prev_states in range(0, N):
                prob = (alpha[prev_states][time - 1] * A[prev_states][states])
                if (prob > prob_mx):
                    target = prev_states
                    prob_mx = prob
            
            predecessor_list[states] = copy.copy(predecessor_list[target])
            predecessor_list[states].append(states)
            
            # multiply it with probabilities to get the observation symbols
            alpha[states].append(prob_mx * B[states][O[time]])
    
    mx_prob = 0
    target = 0
    for states in range(0, N):
        if (alpha[states][T - 1] > mx_prob):
            mx_prob = alpha[states][T - 1]
            target = states
    
    ans = ""
    for a in predecessor_list[target]:
        ans += str(a)
    return ans

In [176]:
most_likely_hmm = hmm_sense(pi, A, B, N, M, T, O)
most_likely_dp = dp_sense(pi, A, B, N, M, T, O)
print("The most likely state sequence according to HMM sense is: " + most_likely_hmm, end = "\n")
print("The most likely state sequence according to DP sense is: " + most_likely_dp, end = "\n")

The most likely state sequence according to HMM sense is: 1010
The most likely state sequence according to DP sense is: 1110


In [177]:
# the procedure to find the gammas
def gammas(pi, A, B, N, M, T, O):
    gamma = [[1] * T for _ in range(0, N)]
    
    # P(O | model)
    P_O = 0
    for i in range(0, N):
        P_O += alpha[i][T - 1]
    
    for time in range(0, T):
        for stat in range(0, N):
            gamma[stat][time] = (alpha[stat][time] * beta[stat][time]) / P_O;
    return gamma

In [178]:
# the procedure to find the digammas
def digammas(pi, A, B, N, M, T, O):
    # Alpha pass and beta pass
    alpha = alpha_pass(pi, A, B, N, M, T, O)
    beta = beta_pass(pi, A, B, N, M, T, O)

    digamma = [[[1] * T for _ in range(0, N)] for _ in range(0, N)]

    # P(O | model)
    P_O = 0
    for i in range(0, N):
        P_O += alpha[i][T - 1]
    
    for time in range(0, T - 1):
        for stat1 in range(0, N):
            for stat2 in range(0, N):
                digamma[stat1][stat2][time] = (alpha[stat1][time] * A[stat1][stat2] * B[stat2][O[time + 1]] * beta[stat2][time + 1]) / P_O;
    
    return digamma

In [179]:
gm = gammas(pi, A, B, N, M, T, O)
dgm = digammas(pi, A, B, N, M, T, O)
print(gm, end="\n\n")
print(dgm)

[[0.1881698097532608, 0.5194317520976989, 0.2288776273157764, 0.8039793968596827], [0.8118301902467394, 0.4805682479023014, 0.7711223726842237, 0.19602060314031736]]

[[[0.14166320511755423, 0.17015867741131513, 0.21080834094874143, 1], [0.04650660463570659, 0.3492730746863837, 0.018069286367034983, 1]], [[0.37776854698014467, 0.05871894990446126, 0.5931710559109413, 1], [0.4340616432665948, 0.42184929799784004, 0.17795131677328238, 1]]]
